# Semantik Arama (Semantic Search)Bu notebook, bns-nlp-engine'in farklı vektör veritabanları ile semantik arama özelliklerini gösterir.

In [ ]:
import asyncioimport osimport timeimport numpy as npfrom bnsnlp.search import FAISSSearch, QdrantSearch, PineconeSearchfrom bnsnlp.embed import HuggingFaceEmbedder, OpenAIEmbedderfrom bnsnlp.preprocess import TurkishPreprocessor

## 1. FAISS - Yerel Vektör AramaFAISS, Facebook'un geliştirdiği yerel vektör arama kütüphanesidir. API key gerektirmez.

In [ ]:
# Embedder oluşturembed der = HuggingFaceEmbedder({    'model': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',    'use_gpu': False})# Test embedding boyutunu altest_result = await embedder.embed("test")dimension = test_result.dimensionsprint(f"Embedding boyutu: {dimension}")

In [ ]:
# FAISS search oluşturfaiss_search = FAISSSearch({    'dimension': dimension,    'index_path': 'faiss_demo.index'})print("✓ FAISS search hazır")

In [ ]:
# Örnek dökümanlardocuments = [    "Python güçlü bir programlama dilidir",    "Makine öğrenmesi yapay zeka alt dalıdır",    "Doğal dil işleme metinleri analiz eder",    "Veri bilimi veriden içgörü çıkarır",    "Derin öğrenme sinir ağları kullanır",    "JavaScript web geliştirme için kullanılır",    "SQL veritabanı sorgulama dilidir",    "Docker konteyner teknolojisidir",    "Kubernetes orkestrasyon platformudur",    "React kullanıcı arayüzü kütüphanesidir"]print(f"{len(documents)} döküman hazırlandı")

In [ ]:
# Embedding'leri oluşturprint("Embedding'ler oluşturuluyor...")start = time.time()embed_result = await embedder.embed(documents)elapsed = time.time() - startprint(f"✓ {len(embed_result.embeddings)} embedding oluşturuldu ({elapsed:.2f}s)")

In [ ]:
# İndeksleids = [str(i) for i in range(len(documents))]metadata = [{'category': 'tech', 'index': i, 'length': len(doc)} for i, doc in enumerate(documents)]await faiss_search.index(documents, embed_result.embeddings, ids, metadata)print(f"✓ {len(documents)} döküman indekslendi")

### Basit Arama

In [ ]:
# Arama yapquery = "yapay zeka ve makine öğrenmesi"# Query embeddingquery_result = await embedder.embed(query)query_embedding = query_result.embeddings[0]# Araresults = await faiss_search.search(query_embedding, top_k=3)print(f"Query: '{query}'\n")print(f"Sonuçlar ({results.query_time_ms:.2f}ms):\n")for i, result in enumerate(results.results, 1):    print(f"{i}. {result.text}")    print(f"   Benzerlik: {result.score:.4f}")    print(f"   ID: {result.id}")    print()

### Farklı Query'ler

In [ ]:
queries = [    "programlama dilleri",    "web teknolojileri",    "veri analizi",    "konteyner ve orkestrasyon"]for query in queries:    query_result = await embedder.embed(query)    results = await faiss_search.search(query_result.embeddings[0], top_k=2)        print(f"Query: '{query}'")    for i, result in enumerate(results.results, 1):        print(f"  {i}. {result.text} (benzerlik: {result.score:.4f})")    print()

## 2. Preprocessing + Search Pipeline

In [ ]:
# Preprocessing eklepreprocessor = TurkishPreprocessor({    'lowercase': True,    'remove_punctuation': True,    'remove_stopwords': True,    'lemmatize': True})# Dökümanları işleraw_documents = [    "Python PROGRAMLAMA dili çok güçlüdür!",    "Makine öğrenmesi, yapay zeka'nın alt dalıdır.",    "Doğal dil işleme metinleri analiz eder."]print("Orijinal dökümanlar:")for i, doc in enumerate(raw_documents, 1):    print(f"{i}. {doc}")# Preprocessingpreprocessed = await preprocessor.process(raw_documents)processed_docs = [p.text for p in preprocessed]print("\nİşlenmiş dökümanlar:")for i, doc in enumerate(processed_docs, 1):    print(f"{i}. {doc}")

In [ ]:
# Yeni index oluşturfaiss_search2 = FAISSSearch({'dimension': dimension})# Embedding ve indekslemeembed_result2 = await embedder.embed(processed_docs)ids2 = [str(i) for i in range(len(processed_docs))]await faiss_search2.index(raw_documents, embed_result2.embeddings, ids2)# Aramaquery = "yapay zeka makine öğrenmesi"query_preprocessed = await preprocessor.process(query)query_embed = await embedder.embed(query_preprocessed.text)results = await faiss_search2.search(query_embed.embeddings[0], top_k=2)print(f"Query: '{query}'")print(f"İşlenmiş query: '{query_preprocessed.text}'\n")for i, result in enumerate(results.results, 1):    print(f"{i}. {result.text}")    print(f"   Benzerlik: {result.score:.4f}\n")

## 3. Qdrant - Cloud Vektör VeritabanıQdrant, yüksek performanslı bir vektör veritabanıdır.

In [ ]:
# Qdrant kontrolüqdrant_url = os.getenv('QDRANT_URL', 'http://localhost:6333')print(f"Qdrant URL: {qdrant_url}")print("\nNot: Qdrant kullanmak için:")print("  1. Docker ile: docker run -p 6333:6333 qdrant/qdrant")print("  2. Cloud: https://cloud.qdrant.io/")

In [ ]:
# Qdrant search (opsiyonel)try:    qdrant_search = QdrantSearch({        'url': qdrant_url,        'collection': 'bns_nlp_demo'    })        # İndeksle    await qdrant_search.index(documents, embed_result.embeddings, ids, metadata)        # Ara    results = await qdrant_search.search(query_embedding, top_k=3)        print("✓ Qdrant arama başarılı\n")    for i, result in enumerate(results.results, 1):        print(f"{i}. {result.text} (benzerlik: {result.score:.4f})")except Exception as e:    print(f"✗ Qdrant bağlantısı başarısız: {e}")    print("  Qdrant sunucusunun çalıştığından emin olun")

## 4. Performans Karşılaştırması

In [ ]:
# Performans testiimport matplotlib.pyplot as plt# Farklı döküman sayılarıdoc_counts = [10, 50, 100, 500, 1000]index_times = []search_times = []for count in doc_counts:    test_docs = [f"Test döküman {i}" for i in range(count)]        # Index time    start = time.time()    test_embeddings = await embedder.embed(test_docs)    test_search = FAISSSearch({'dimension': dimension})    test_ids = [str(i) for i in range(count)]    await test_search.index(test_docs, test_embeddings.embeddings, test_ids)    index_time = time.time() - start    index_times.append(index_time)        # Search time    start = time.time()    await test_search.search(query_embedding, top_k=10)    search_time = time.time() - start    search_times.append(search_time)        print(f"{count:4d} döküman: index={index_time:.3f}s, search={search_time*1000:.2f}ms")# Grafikfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))ax1.plot(doc_counts, index_times, marker='o', linewidth=2, markersize=8)ax1.set_xlabel('Döküman Sayısı', fontsize=12)ax1.set_ylabel('İndeksleme Süresi (s)', fontsize=12)ax1.set_title('İndeksleme Performansı', fontsize=14)ax1.grid(True, alpha=0.3)ax2.plot(doc_counts, [t*1000 for t in search_times], marker='o', linewidth=2, markersize=8, color='orange')ax2.set_xlabel('Döküman Sayısı', fontsize=12)ax2.set_ylabel('Arama Süresi (ms)', fontsize=12)ax2.set_title('Arama Performansı', fontsize=14)ax2.grid(True, alpha=0.3)plt.tight_layout()plt.show()

## 5. Gerçek Dünya Örneği: Haber Arama Sistemi

In [ ]:
# Örnek haber metinlerinews_articles = [    "Türkiye'de yapay zeka teknolojileri hızla gelişiyor. Yerli şirketler önemli projeler üretiyor.",    "Ekonomi alanında yeni düzenlemeler yapıldı. Merkez Bankası faiz kararını açıkladı.",    "Spor dünyasında heyecan verici gelişmeler yaşanıyor. Milli takım önemli bir galibiyet aldı.",    "Eğitim sisteminde dijital dönüşüm hızlanıyor. Uzaktan eğitim platformları yaygınlaşıyor.",    "Sağlık sektöründe teknoloji kullanımı artıyor. Yapay zeka destekli teşhis sistemleri geliştiriliyor.",    "İklim değişikliği ile mücadele kapsamında yeni projeler başlatıldı. Yenilenebilir enerji yatırımları artıyor.",    "Teknoloji şirketleri yeni ürünlerini tanıttı. Yapay zeka ve makine öğrenmesi ön planda.",    "Otomotiv sektöründe elektrikli araç üretimi hızlanıyor. Yerli otomobil projesi ilerliyor."]print(f"{len(news_articles)} haber makalesi")

In [ ]:
# Haberleri işle ve indekslepreprocessed_news = await preprocessor.process(news_articles)processed_news = [p.text for p in preprocessed_news]news_embeddings = await embedder.embed(processed_news)news_search = FAISSSearch({'dimension': dimension})news_ids = [str(i) for i in range(len(news_articles))]news_metadata = [{'category': 'news', 'index': i} for i in range(len(news_articles))]await news_search.index(news_articles, news_embeddings.embeddings, news_ids, news_metadata)print("✓ Haberler indekslendi")

In [ ]:
# Farklı sorguları test ettest_queries = [    "yapay zeka teknolojileri",    "ekonomi ve finans",    "spor haberleri",    "eğitim ve teknoloji",    "sağlık sektörü"]for query in test_queries:    # Query'yi işle    query_proc = await preprocessor.process(query)    query_emb = await embedder.embed(query_proc.text)        # Ara    results = await news_search.search(query_emb.embeddings[0], top_k=2)        print(f"\nQuery: '{query}'")    print("-" * 80)    for i, result in enumerate(results.results, 1):        print(f"{i}. {result.text[:100]}...")        print(f"   Benzerlik: {result.score:.4f}")

## Sonraki Adımlar- [Quickstart Notebook](quickstart.ipynb) - Hızlı başlangıç- [Preprocessing Notebook](preprocessing.ipynb) - Metin ön işleme- [Embeddings Notebook](embeddings.ipynb) - Embedding oluşturma- [API Dokümantasyonu](https://yourusername.github.io/bns-nlp-engine/api/search/)